This explanation of random incidence in time to stake presumes some basic knowledge of how Proof-of-Stake cryptocurrencies operate. For more information, see [this paper](https://www.researchgate.net/publication/357510506_Expected_Time_to_Stake_and_Net_Weight_in_Proof-of-Stake_Cryptocurrencies). 

For the purposes of this explanation, all UTXOs that are mentioned should be assumed to be online and attempting to stake blocks, that the pool of UTXOs does not change, and the effects of cooldown are ignored. 

Consider some UTXO. During every block formation, that UTXO has a probability of staking $p$ equal to $\frac{\text{value of UTXO}}{\text{value of all UTXOs}}$, and the repeated trials constitute a Bernoulli process with parameter $p$. This implies that the probability mass function of the number of trials (block formations) between one stake and another is geometric with parameter $p$. Due to the memorylessness of the geometric distribution, this will be true for every *inter-stake interval*, which is the time between stakes, and in this case is measured in units of blocks. For the purposes of this explanation, we will refer to inter-stake intervals as *inter-arrival times*, which is the commonly used terminology.

This means that for the UTXO, the expected inter-arrival time is $\frac{1}{p}$, since the distribution is geometric. However, from the point of view of the owner of the UTXO, the expected *observed* inter-arrival time will be almost double the expected inter-arrival time, due to a phenomenon called *random incidence*. 

--------------------------------

To understand this intuitively, consider the following example. Your friend is sitting in a room, flipping a coin that lands on heads with some probability $p_c$; let's say for simplicity that $p_c = \frac{1}{2}$. If the coin lands on heads, then your friend waits 99 minutes, and then flips the coin again. If the coin lands on tails, then your friend waits 1 minute, and then flips the coin again. From the perspective of your friend, the average waiting time will be $p_c \cdot 99 + p_c \cdot 1 = 50$. 

Now suppose that you open the door, and observe your friend during one of the waiting periods. What is the expected waiting time that you observe? Well, $99\%$ of the time, your friend will be in a waiting period of $99$ minutes, and $1\%$ of the time, your friend will be in a waiting period of $1$ minute. Thus, the expected waiting time you will see is $0.99 \cdot 99 + 0.01 \cdot 1 = 98.02$ minutes, which is almost twice as long as the average waiting time of $50$ minutes that your friend experiences.

This phenomenon is called random incidence. [Here's someone much smarter than me explaining the same concept](https://www.youtube.com/watch?v=aS1o7uTaLF0).

A very similar phenomenon occurs when staking. If you randomly observe a UTXO while it is trying to stake, you are much more likely to observe a time interval between stakes that is longer than the average.

--------------------------------

Returning to the actual scenario, since the number of trials/blocks it takes to stake is geometrically distributed, the support of the distribution is $\mathbb{Z}^+$. That is, unlike in the example just cited, it can take any positive integer number of trials to finally stake.  

Note: the following observation has been adapted from the MITx course [Probability - The Science of Uncertainty and Data](https://www.edx.org/course/probability-the-science-of-uncertainty-and-data).

As already mentioned, the expected inter-arrival time is $\frac{1}{p}$. However, the expected observed inter-arrival time is different. Let $X$ denote the random variable corresponding to the number of blocks/trials until a stake, and let $t_i$ denote $i$ number of blocks/trials. Then probability of the UTXO taking $t_i$ blocks/trials to stake is

$$
P (X = t_i) = (1 - p)^{t_i - 1} \cdot p
$$


Let $T$ denote the random variable corresponding to the *observed* number of blocks/trials until a stake. In order to build an intuition for the following calculation, suppose that all inter-arrival times are equally likely, and that there are $1, 2, \ldots, n$ possible interarrival times. In that case, 

$$
P(T = t_i) = \frac{t_i}{\sum_{j = 1}^n t_j}
$$

That is, the probability of seeing an interval of a particular length would be the length of that interval divided by the sum of the lengths of all intervals. However, in this case, not all intervals have an equal probability of occuring. To account for this, we need to weight the intervals by their probabilities of occuring.


$$
P (T = t_i) = 
\frac{t_i \cdot P(X = t_i)}{\sum_{j = 1}^{\infty} t_j \cdot P(X = t_j)}
= \frac{t_i \cdot P(X = t_i)}{\mathbb{E}[T]} 
$$

Then, the expected number of blocks/trials until staking that an observer will see is 

$$
\sum_{i = 1}^{\infty} t_i \cdot P (T = t_i)
= \sum_{i = 1}^{\infty} t_i \frac{t_i \cdot P(X = t_i)}{\mathbb{E}[T]}
= \frac{\sum_{i = 1}^{\infty} t_i^2 \cdot P(X = t_i)}{\mathbb{E}[T]} 
= \frac{\mathbb{E}[ T^2 ]}{\mathbb{E}[ T ]} 
$$

As an example, suppose that you have a UTXO corresponding to 100,000 coins, and there is a total of 100,000,000 coins online and attempting to stake. The expected number of blocks/trials to stake would be:

In [1]:
p = 10**5 / 10**8
expected_interarrival_time = 1 / p 
expected_interarrival_time

1000.0

On the other hand, if we use the formula for the expected number of trials/blocks that an observer will see (taking a partial sum as an approximation), 

In [2]:
relevant_sum1 = sum([i**2 * (1 - p)**(i - 1) * p for i in range(1000000)])
relevant_sum2 = sum([i * (1 - p)**(i - 1) * p for i in range(1000000)])
observed_expected_interarrival_time1 = relevant_sum1 / relevant_sum2
observed_expected_interarrival_time1

1998.9999999999839

The ratio of the expected inter-interval time to the observed expected inter-interval is

In [3]:
observed_expected_interarrival_time1 / expected_interarrival_time

1.998999999999984

which confirms our earlier claim of the observed expected time to stake being approximately twice the expected time to stake.

We can also simulate this same phenomenon. First, we will take a random sample of inter-arrival times.

In [4]:
import numpy as np

n = 10000000
arr = np.random.geometric(p=p, size=n)
arr

array([1184,  169, 2092, ..., 1960,  848,  433])

The sample average inter-arrival time is

In [5]:
sample_expected_interarrival_time = arr.mean()
sample_expected_interarrival_time

999.7903209

<!-- 
The sample average inter-arrival time that a person sees is the sum of each staking time $\times$ that staking time divided by the sum of all staking times. 
-->

The sample average inter-arrival time is the sum of the product of each inter-arrival time with its fraction of all inter-arrival times.

In [6]:
sample_observed_expected_interarrival_time2 = np.dot(arr, (arr / np.sum(arr)))
sample_observed_expected_interarrival_time2

1998.6028556983388

The ratio of the sample expected inter-interval time to the sample observed expected inter-interval is

In [7]:
sample_observed_expected_interarrival_time2 / sample_expected_interarrival_time

1.9990220088340314

We can also determine the answer analytically:

$$
E[T] 
= \frac{1}{p}
$$

and

$$
E[T^2] 
= \frac{1 - p}{p^2} + \left( \frac{1}{p} \right)^2
$$

Implying that

$$
\frac{E[T^2]}{E[T]} 
= \frac{\frac{1 - p}{p^2} + (\frac{1}{p})^2}{\frac{1}{p}}
= \frac{1 - p}{p} + \frac{1}{p}
= \frac{2 - p}{p}
= \frac{2}{p} - 1
$$

Calculating this,

In [8]:
observed_expected_interarrival_time3 = 2 / p - 1
observed_expected_interarrival_time3

1999.0

The ratio of the analytic expected inter-interval time to the analytic observed expected inter-interval is

In [9]:
observed_expected_interarrival_time3 / expected_interarrival_time

1.999

... and we see that the analytic answer conforms with the raw formula approach as well as the simulation approach.